In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [5]:
covidNA = pd.read_csv('coviddata/casesNA.csv',compression = 'gzip')
#traffic = pd.read_csv("airport_traffic_NA.csv")
#coviddata/casesCanada.csv, casesUS.csv

In [6]:
covidNA

,Active,Case-Fatality_Ratio,Combined_Key,Confirmed,Country/Region,Deaths,Incidence_Rate,Last Update,Latitude,Longitude,Province/State,Recovered
0,2836.0,1.322618,"Alberta, Canada",21775,Canada,288,493.412183,2020-10-18 04:24:25,53.933300,-116.576500,Alberta,18651
1,1513.0,2.243275,"British Columbia, Canada",11189,Canada,251,218.923532,2020-10-18 04:24:25,53.726700,-127.647600,British Columbia,9425
2,0.0,0.000000,"Grand Princess, Canada",13,Canada,0,NaN,2020-10-18 04:24:25,NaN,NaN,Grand Princess,13
3,1582.0,1.197605,"Manitoba, Canada",3173,Canada,38,230.341985,2020-10-18 04:24:25,53.760900,-98.813900,Manitoba,1553
4,92.0,0.673401,"New Brunswick, Canada",297,Canada,2,38.077265,2020-10-18 04:24:25,46.565300,-66.461900,New Brunswick,203
5,12.0,1.393728,"Newfoundland and Labrador, Canada",287,Canada,4,55.047807,2020-10-18 04:24:25,53.135500,-57.660400,Newfoundland and Labrador,271
6,0.0,0.000000,"Northwest Territories,Canada",5,Canada,0,11.134865,2020-10-18 04:24:25,64.825500,-124.845700,Northwest Territories,5
7,4.0,5.946935,"Nova Scotia, Canada",1093,Canada,65,111.820776,2020-10-18 04:24:25,44.682000,-63.744300,Nova Scotia,1024
8,6013.0,4.691129,"Ontario, Canada",65869,Canada,3090,447.728212,2020-10-18 04:24:25,51.253800,-85.323200,Ontario,56766
9,3.0,0.000000,"Prince Edward Island, Canada",63,Canada,0,39.833584,2020-10-18 04:24:25,46.510700,-63.416800,Prince Edward Island,60
